First cell is necessary for running this notebook with django env (as if it was executed from an API call).

In [1]:
import os, sys
DJANGO_ROOT = os.path.abspath(os.path.join(os.getcwd(),'../../'))
sys.path.insert(0,DJANGO_ROOT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [9]:
from app.models import Profile, Image, LikedImages
import cPickle as pckl

# Like image

In [502]:
data = {'like': False, 'id':1 }

In [503]:
user = User.objects.all().first()

In [504]:
profile = Profile.objects.all().first()

In [505]:
# import numpy as np
# profile.pref_vec = pckl.dumps( np.zeros_like(pckl.loads(str(profile.pref_vec))))
# profile.img_count = 0
# profile.save()

In [506]:
query_res = LikedImages.objects.filter(user= user.id, img=data['id'] )

if data['like']:
    # Add the image to list if it's not there yet
    if len(query_res) is 0:
        image = Image.objects.filter(id=data['id']).first()

        # update user preferences
        profile = Profile.objects.filter(user_id=user.id).first()
        profile_vec = pckl.loads(str(profile.pref_vec))
        img_vec     = pckl.loads(str(image.txt_vec))

        profile_vec = ((profile_vec * profile.img_count) + img_vec) / (profile.img_count + 1 )
        profile.img_count += 1
        profile.pref_vec = pckl.dumps(profile_vec)
        profile.save()

        # save like
        li = LikedImages(user=user, img=image)
        li.save()
else:
    if len(query_res) is 1:
        result = query_res.first()
        
        # update user preferences
        profile = Profile.objects.filter(user_id=user.id).first()
        profile_vec = pckl.loads(str(profile.pref_vec))
        img_vec     = pckl.loads(str(result.img.txt_vec))

        profile_vec = ((profile_vec * profile.img_count) - img_vec)
        profile_vec = profile_vec if profile.img_count == 1 else profile_vec / (profile.img_count - 1 )
        profile.img_count -= 1
        profile.pref_vec = pckl.dumps(profile_vec)
        profile.save()
        
        result.delete()

In [507]:
len(LikedImages.objects.all())

0

Test for succ user update:

In [508]:
user = Profile.objects.all().first()

In [509]:
pckl.loads(str(user.pref_vec))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [510]:
Profile.objects.first().img_count

0L

# Retrieve recommendations

In [527]:
images = Image.objects.all()

In [528]:
user = User.objects.all().first()
profile = Profile.objects.filter(user_id = user.id).first()

In [529]:
pref_vec = pckl.loads(str(profile.pref_vec))

In [530]:
sim = []
for img in images:
    img_vec = pckl.loads(str(img.txt_vec)) 
    similarity = np.arccos(np.dot(img_vec,pref_vec) / (np.linalg.norm(img_vec) * np.linalg.norm(pref_vec))) 
    sim.append( (img.id,similarity))

In [531]:
sim.sort(key=lambda tup: tup[1])

In [532]:
top_k = 16
indices = map(lambda rec: rec[0], sim[:top_k])

In [533]:
indices

[9L,
 2L,
 4529L,
 4339L,
 1427L,
 1884L,
 2752L,
 1679L,
 1575L,
 3261L,
 4791L,
 4267L,
 12L,
 4298L,
 3813L,
 1592L]

In [536]:
Image.objects.filter(id__in = indices)

16